### Установим нужные библиотеки

In [ ]:
%pip install pandas 
%pip install open3d 
%pip install "laspy[lazrs,laszip]"
%pip install "kiss-icp[all]"

In [1]:
import os
import time
import laspy
import numpy as np
import pandas as pd
import open3d as o3d

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


### Попробуем отрисовать 3D сцену

In [2]:
pc_path = '/testdata/macchiato_datasets/tst-mac-10-2023-2/new.002/new.002.018.robosenseCapture'
pc_fn = 'new.002.018.robosenseCapture.000000.laz'
pc1 = laspy.read(os.path.join(pc_path, pc_fn))

In [ ]:
xyz = np.vstack((pc1.x, pc1.y, pc1.z)).transpose()
xyz.shape # 57600 точек

In [ ]:
intensity = pc1.intensity / 255
rgb = np.vstack((intensity, 1 - intensity, 0 * intensity)).transpose()
rgb.shape

In [ ]:
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(xyz)
pcd.colors = o3d.utility.Vector3dVector(rgb)
print('pcd:', pcd)

In [ ]:
o3d.visualization.draw_geometries([pcd])

#### Работа с множеством сцен и визуализация

In [5]:
from tqdm import tqdm

# Загрузим все измерения из папки data
points = []

# pc_path = '/home/nikita/testdata/upload/xtrail_20230914/20230914/new.152524.002/robosenseCapture'
pc_path_out = './data'
files_list = os.listdir(pc_path)
files_list.sort()

for file in tqdm(files_list):
    pc = laspy.read(os.path.join(pc_path, file))
    xyz = np.vstack((pc.x, pc.y, pc.z)).transpose()
    points.append(xyz)
    np.savetxt(os.path.join(pc_path_out, file[:-4] + '.xyz'), xyz)

points = np.array(points)

100%|██████████| 99/99 [00:14<00:00,  6.86it/s]


#### Модуль kiss-icp очень капризный к формату данных сконвертируем все в сырой .xyz

In [6]:
# Рассчет матриц перехода для свех наших измерений
!kiss_icp_pipeline ./data 

Trying to guess how to read your data: `pip install "kiss-icp[all]"` is required
100%|██████████████████████████████████████| 99/99 [00:09<00:00, 10.54 frames/s]
[WARNING] No GT poses available, skipping evaluation


In [7]:
poses = np.load('./results/latest/data_poses.npy') # Матрицы перехода для однородных координат

In [8]:
transformed_points = np.zeros((0, 3))
col_len = np.ones(points[0].shape[0])
for i in range(len(points)):
    hom_points = np.c_[points[i], col_len]
    hom_points = np.dot(poses[i], hom_points.transpose()).transpose()
    het_points = hom_points[:, :-1] * np.column_stack((hom_points[:, -1], hom_points[:, -1], hom_points[:, -1]))
    transformed_points = np.append(transformed_points, het_points, axis=0)

transformed_points.shape

(5702400, 3)

In [9]:
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(transformed_points)
o3d.visualization.draw_geometries([pcd])